In [3]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

sc = spark.sparkContext

In [8]:
months = ("JAN","FEB","MAR","APR","MAY","JUN","JUL")
monthsRdd = sc.parallelize(months)
monthsIndexed0Rdd = monthsRdd.zipWithIndex()

print(monthsIndexed0Rdd.collect())

[('JAN', 0), ('FEB', 1), ('MAR', 2), ('APR', 3), ('MAY', 4), ('JUN', 5), ('JUL', 6)]


In [9]:
monthsIndexed1Rdd = monthsIndexed0Rdd.map(lambda x: (x[0], x[1] + 1))
 
print(monthsIndexed1Rdd.collect())

[('JAN', 1), ('FEB', 2), ('MAR', 3), ('APR', 4), ('MAY', 5), ('JUN', 6), ('JUL', 7)]


In [10]:
monthsIndexed2Rdd = monthsIndexed1Rdd.mapValues(lambda y: y + 1)

print(monthsIndexed2Rdd.collect())

[('JAN', 2), ('FEB', 3), ('MAR', 4), ('APR', 5), ('MAY', 6), ('JUN', 7), ('JUL', 8)]


In [11]:
quarters = (1, 1, 1, 2, 2, 2, 3)
quartersRdd = sc.parallelize(quarters)
monthsZipQuarters = monthsRdd.zip(quartersRdd)

print(monthsZipQuarters.collect())

[('JAN', 1), ('FEB', 1), ('MAR', 1), ('APR', 2), ('MAY', 2), ('JUN', 2), ('JUL', 3)]


In [12]:
print(monthsZipQuarters.keys().collect())
print(monthsZipQuarters.values().collect())
print(monthsZipQuarters.sortByKey().collect())


['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL']
[1, 1, 1, 2, 2, 2, 3]
[('APR', 2), ('FEB', 1), ('JAN', 1), ('JUL', 3), ('JUN', 2), ('MAR', 1), ('MAY', 2)]


In [13]:
splitRdd = sc.textFile('file:////home/talentum/test-jupyter/test/PairRdd/selfishgiant.txt').flatMap(lambda line: line.split(' '))
print(splitRdd.take(5))

['EVERY', 'afternoon,', 'as', 'they', 'were']


In [14]:
mappedRdd = splitRdd.map(lambda word: (word,1))

print(mappedRdd.take(5))

[('EVERY', 1), ('afternoon,', 1), ('as', 1), ('they', 1), ('were', 1)]


In [15]:
reducedByKeyRdd = mappedRdd.reduceByKey(lambda x,y : x+y)

print(reducedByKeyRdd.take(5))

[('EVERY', 1), ('afternoon,', 3), ('as', 9), ('they', 17), ('were', 21)]


In [16]:
flippedRdd = reducedByKeyRdd.map(lambda x: (x[1], x[0]))

print(flippedRdd.take(5))

[(1, 'EVERY'), (3, 'afternoon,'), (9, 'as'), (17, 'they'), (21, 'were')]


In [17]:
orderedRdd = flippedRdd.sortByKey(ascending=False)

print(orderedRdd.take(5))

[(148, 'the'), (85, 'and'), (44, 'he'), (38, 'to'), (33, '')]
